In [ ]:
# Install cirq and openfermion for use in google Colab
!pip install --quiet cirq;
!pip install --quiet openfermion;
import cirq
import openfermion
import matplotlib.pyplot as plt
import sympy
import numpy as np

**Write a function to generate the Hamiltonian of the H2 molecule**

In [ ]:
"From tutorial #4, picking an arbiatry H2 molecule and getting its Hamiltonian from OpenFermion"

diatomic_bond_length = .7414

geometry = [('H', (0., 0., 0.)), 
            ('H', (0., 0., diatomic_bond_length))]

basis = 'sto-3g'
multiplicity = 1
charge = 0
description = str(diatomic_bond_length)

molecule = openfermion.MolecularData(
    geometry=geometry,
    basis=basis,
    multiplicity=multiplicity,
    description=description)
molecule.load()

print(molecule.name)

H2_sto-3g_singlet_0.7414


In [ ]:
"Turning the Hamiltonian from an interaction_operator class to a sparse_operator_class. This is neccesary for many transformations."

hamiltonian = molecule.get_molecular_hamiltonian()
print(hamiltonian.__class__)

sparse_hamiltonian = openfermion.get_sparse_operator(hamiltonian)

<class 'openfermion.ops.representations.interaction_operator.InteractionOperator'>


General Fermionic Hamiltonian:

 $H = Σ_{i=0,1,2,3}ϵ_i a^†_i a_i + Σ_{i=0,1,2,3}Σ_{j=0,1,2,3} V_{ij}a^†_ja^†_i a_j a_i = H_0 + H_{int}$ 

Where $H_{int}$ are the non-quadratic terms, which can be treated as a perturbation because $V$ is generally small. 

In [ ]:
"""Sparse Hamiltonian -> fermionic Hamiltonian -> non-int fermionic Hamiltonian.
It is neccesary to turn the Hamiltonian into the 2nd quantized form. Afterwards we will Jordan-Wigner.
Note that only the non-interacting (quadratic) terms are needed to find the mean-field (HF) ground state"""
#Note: We do not need to parameterize all terms in fermionic_hamiltonian
fermionic_hamiltonian = openfermion.get_fermion_operator(hamiltonian)

terms = list(fermionic_hamiltonian.terms)
coeffs = list(fermionic_hamiltonian.terms.values())

ham = openfermion.FermionOperator(terms[0], coeffs[0]) #The identity contribution

for i in range(1,len(terms)): 
  if len(terms[i]) == 2:
    ham += openfermion.FermionOperator(terms[i], coeffs[i].real)

print(ham)
non_int_hamiltonian = openfermion.get_sparse_operator(ham)

0.713753990544915 [] +
-1.2524635715927757 [0^ 0] +
-1.2524635715927757 [1^ 1] +
-0.4759487172683097 [2^ 2] +
-0.4759487172683097 [3^ 3]


**Write a function to generate the Trotterized parameterized Hamiltonian ansatz.**

For this ansatz I am starting explicitly from the HF state. To find this we would use a generate VQE for H2 with ~26 parameters, under controlled time evolution for the quadratic (non-interacting) Hamiltonian. I have not done this explicitly here to try and be concise.

https://pennylane.ai/qml/demos/tutorial_givens_rotations.html

https://arxiv.org/pdf/2106.13839.pdf

In [ ]:
n_qubits = openfermion.count_qubits(hamiltonian)
qubits = cirq.LineQubit.range(n_qubits)

In [ ]:
def double_exc_unitary(theta):
  unitary = np.identity(16)
  unitary[3][3] = np.cos(theta/2)
  unitary[3][12] = -np.sin(theta/2)
  unitary[12][3] = np.sin(theta/2)
  unitary[12][12] = np.cos(theta/2)
  return unitary

"""Define a custom gate with a parameter."""
class DoubleExcGate(cirq.Gate):
    def __init__(self, theta):
        super(DoubleExcGate, self)
        self.theta = theta

    def _num_qubits_(self):
        return 1

    def _unitary_(self):
        return double_exc_unitary(self.theta) / 1

    def _circuit_diagram_info_(self, args):
        return [f"G^2({round(self.theta,2)})"] * self.num_qubits()


In [ ]:
def state_preparation(params):
  ansatz_circ = cirq.Circuit(cirq.X(qubits[0]), cirq.X(qubits[1]))
  ansatz_circ.append(cirq.ops.givens(params[0]).on(qubits[1],qubits[2]))
  ansatz_circ.append(cirq.ops.givens(params[1]).on(qubits[1],qubits[3]))
  ansatz_circ.append(cirq.ops.givens(params[2]).on(qubits[0],qubits[2]).controlled_by(qubits[1]))
  ansatz_circ.append(cirq.ops.givens(params[3]).on(qubits[0],qubits[3]).controlled_by(qubits[1]))
  ansatz_circ.append(DoubleExcGate(params[4]).on(*qubits))
  return ansatz_circ

n = 6
params = np.array([-2 * np.arcsin(1/np.sqrt(n-i)) for i in range(n-1)])

print(state_preparation(params))
print(cirq.dirac_notation(state_preparation(params).final_state_vector(0)))

0: ───X────────────────────────────────────────────────PhISwap(0.25)^(-2/3)───PhISwap(0.25)^-0.784───G^2(-1.57)───
                                                       │                      │                      │
1: ───X───PhISwap(0.25)──────────PhISwap(0.25)─────────@──────────────────────@──────────────────────G^2(-1.57)───
          │                      │                     │                      │                      │
2: ───────PhISwap(0.25)^-0.535───┼─────────────────────PhISwap(0.25)──────────┼──────────────────────G^2(-1.57)───
                                 │                                            │                      │
3: ──────────────────────────────PhISwap(0.25)^-0.59──────────────────────────PhISwap(0.25)──────────G^2(-1.57)───
0.05|0011⟩ + 0.19|0101⟩ + 0.35|0110⟩ + 0.53|1001⟩ + 0.75|1010⟩ + 0.05|1100⟩


**Appropriate value if t based on non-interacting part of Hamiltonian.**

In [ ]:
print('Eigenspectrum of H2 in FermionOperator: \n',
      openfermion.eigenspectrum(fermionic_hamiltonian))
print("Hartree Fock (mean-field) energy in Hartrees:", molecule.hf_energy)

Eigenspectrum of H2 in FermionOperator: 
 [-1.13727017 -0.53870958 -0.53870958 -0.53247901 -0.53247901 -0.53247901
 -0.44698572 -0.44698572 -0.16990139  0.23780527  0.23780527  0.35243413
  0.35243413  0.47983611  0.71375399  0.92010671]
Hartree Fock (mean-field) energy in Hartrees: -1.116684386906734


In [ ]:
def cost_fun_non_int(result):
  final_state = result.final_state_vector
  return openfermion.expectation(non_int_hamiltonian, final_state)

def cost_fun_int(result):
  final_state = result.final_state_vector
  return openfermion.expectation(sparse_hamiltonian, final_state).real

In [ ]:
from scipy.optimize import minimize

coeffs = np.full(5, 1)
simulator = cirq.Simulator()

def sp_cost_function(coeffs):
  circuit = state_preparation(coeffs)
  result = simulator.simulate(circuit, initial_state=0)
  e_tot = cost_fun_non_int(result).real
  return e_tot

opt_res = minimize(sp_cost_function, coeffs, method='cobyla')
print(opt_res)

mf_coeffs = opt_res.x
print('mean-field ground-state:',cirq.dirac_notation(state_preparation(mf_coeffs).final_state_vector(0)))

print('mean-field energy:',cost_fun_int(simulator.simulate(cirq.Circuit(cirq.ops.identity_each(*qubits)), initial_state=12)))

     fun: -1.7911728495941022
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 94
  status: 1
 success: True
       x: array([-4.71470140e-04,  7.68947237e-05, -1.75828321e-04,  2.23058919e-04,
        6.36654997e-04])
mean-field ground-state: |1100⟩
mean-field energy: -1.116684386906734


**Write a function to generate a circuit for time evolution under the full H_2 Hamiltonian for a time *t*, conditional on an ancilla qubit.**

In [ ]:
n_qubits = openfermion.count_qubits(hamiltonian)

qubits = cirq.LineQubit.range(n_qubits)
control = cirq.LineQubit(-1)

In [ ]:
from openfermion.circuits import trotter

# Define a time evolution circuit.
def controlled_trot(trot_time, trot_steps):
  yield openfermion.simulate_trotter(
    qubits, hamiltonian, trot_time,
    n_steps=trot_steps,
    order=0,
    algorithm=trotter.LOW_RANK,
    control_qubit=control)

def waves1_circ(coeffs, trot_time, trot_steps=1, exc_circ=cirq.Circuit(cirq.ops.identity_each(*qubits))):
    yield cirq.H(control)
    yield state_preparation(coeffs)
    yield exc_circ
    yield controlled_trot(trot_time, trot_steps)

time = 1

# Construct phase estimation circuit.
circuit = cirq.Circuit(waves1_circ(mf_coeffs, time))

# Print the circuit.
cirq.DropNegligible().optimize_circuit(circuit)
print(circuit.to_text_diagram(transpose=False))

                                                                                                                                                                                                                                         ┌─────────┐                                   ┌──────────┐                   ┌─────────┐                                                                                                                                                                                                                   ┌─────────────┐                                                                                                        ┌─────────┐                               ┌────────┐                 ┌─────────┐
-1: ───H──────────────────────────────────────────────────────────────────────────────────────────────────@─────────@─────────@─────────@─────────────────────────────────────────────────────────────────────────────────────@────────────@───────────────@───────────────@

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: DropNegligible was used but is deprecated.
It will be removed in cirq v1.0.
Use cirq.drop_negligible_operations instead.



In [ ]:
def control_expval(coeffs, trot_time, label, n_shots, trot_steps=1, exc_circ=cirq.Circuit(cirq.ops.identity_each(*qubits))):
  circuit = cirq.Circuit(waves1_circ(coeffs, trot_time, trot_steps, exc_circ)) # copy the circuit
  # change bases
  if label == 'X':
    circuit.append(cirq.H(control))
  elif label == 'Y':
    circuit.append(cirq.rx(np.pi/2)(control))
  elif label == 'Z':
    pass
  circuit.append(cirq.measure(control, key=label))
  res = simulator.run(circuit, repetitions=int(n_shots))
  hist = res.histogram(key=label)
  return (hist[0] - hist[1]) / n_shots

control_expval(mf_coeffs, time, 'X', 1e4)

0.4326

In [ ]:
#Making sure that it works for all times (or at least t ~ [0, 2pi]) using division modulo %
mu = np.max(np.abs(openfermion.eigenspectrum(fermionic_hamiltonian))) #Bound the t by the largest eigenvalue
time = 1

#Create the reduced density matrix
def tomo_DM(coeffs, trot_time, n_shots, trot_steps=1, exc_circ=cirq.Circuit(cirq.ops.identity_each(*qubits))):
  DM = np.array(cirq.unitary(cirq.I), dtype=complex)
  exp_X = control_expval(coeffs, trot_time % ((np.pi)/mu), 'X', n_shots, trot_steps, exc_circ)
  exp_Y = control_expval(coeffs, trot_time % ((np.pi)/mu), 'Y', n_shots, trot_steps, exc_circ)
  #exp_Z = control_expval(coeffs, trot_time % ((np.pi)/mu), 'Z', n_shots, trot_steps, exc_circ)
  #DM[0,0] += exp_Z
  #DM[1,1] += -exp_Z
  DM[0,1] += exp_X - 1j*exp_Y
  DM[1,0] += exp_X + 1j*exp_Y
  return DM*0.5

rho_c = tomo_DM(mf_coeffs, time, 1e4)
print(rho_c)

[[0.5   +0.j     0.2136-0.4415j]
 [0.2136+0.4415j 0.5   +0.j    ]]


In [ ]:
#Check its accuracy
final_state = cirq.Circuit(waves1_circ(mf_coeffs, time)).final_state_vector()
final_control_state = cirq.linalg.partial_trace_of_state_vector_as_mixture(final_state, [0])

rho_c_exact = np.zeros((2,2), dtype=complex)

for i in range(len(final_control_state)):
  rho_c_exact += final_control_state[i][0]*cirq.qis.density_matrix_from_state_vector(final_control_state[i][1])

print(rho_c_exact)

[[0.5       +0.j         0.21572486-0.44200014j]
 [0.21572486+0.44200014j 0.5       +0.j        ]]


In [ ]:
print('Energy evaluated from off diagonal elements of rho_c:', -abs(np.angle(rho_c[0,1])))

final_target_state = cirq.linalg.partial_trace_of_state_vector_as_mixture(final_state, [1,2,3,4])
psi_t_exact = np.zeros(16, dtype=complex)
for i in range(len(final_target_state)):
  psi_t_exact += final_target_state[i][0]*final_target_state[i][1]
print('Exact energy:', openfermion.expectation(sparse_hamiltonian, psi_t_exact).real)

Energy evaluated from off diagonal elements of rho_c: -1.1201882851048464
Exact energy: -1.0835724562579412


In [ ]:
#Making sure that it works for all times (or at least t ~ [0, 2pi])
mu = np.max(np.abs(openfermion.eigenspectrum(fermionic_hamiltonian)))
time_step = 5

for i in range(1, 10):
  time = i*time_step
  rho_c = tomo_DM(mf_coeffs, time, 1e4)
  print('Energy evaluated from off diagonal elements of rho_c at time t = {}:'.format(time), -abs(np.angle(rho_c[0,1])/(time % ((np.pi)/mu))))

print('Exact energy:', openfermion.expectation(sparse_hamiltonian, psi_t_exact).real)

Energy evaluated from off diagonal elements of rho_c at time t = 5: -1.1100612721848406
Energy evaluated from off diagonal elements of rho_c at time t = 10: -1.116355059135117
Energy evaluated from off diagonal elements of rho_c at time t = 15: -1.1165397014234804
Energy evaluated from off diagonal elements of rho_c at time t = 20: -1.1258822332464895
Energy evaluated from off diagonal elements of rho_c at time t = 25: -1.0493241571739185
Energy evaluated from off diagonal elements of rho_c at time t = 30: -1.1097316390472969
Energy evaluated from off diagonal elements of rho_c at time t = 35: -1.1260501513842365
Energy evaluated from off diagonal elements of rho_c at time t = 40: -1.116877894556007
Energy evaluated from off diagonal elements of rho_c at time t = 45: -1.1132100877147184
Exact energy: -1.0835724562579412


In [ ]:
#tests
time = 1

def purity(density_matrix):
  return np.trace(np.matmul(density_matrix, density_matrix)).real

print('Purity of reduced DM for the control qubit (state tomo):', purity(tomo_DM(mf_coeffs, time, 1e4)))
print('Purity of reduced DM for the control qubit (exact):', purity(rho_c_exact))

#Trotter error from one run cancels out with many samples

Purity of reduced DM for the control qubit (state tomo): 0.9860781
Purity of reduced DM for the control qubit (exact): 0.9838026818095387


**Combine the above functions to implement step 1 of the WAVES protocol**

In [ ]:
def cost_fun_tomo(coeffs, trot_time, n_shots, Temp, trot_steps=1, exc_circ=cirq.Circuit(cirq.ops.identity_each(*qubits))):
  rho_c = tomo_DM(coeffs, trot_time, n_shots, trot_steps, exc_circ)
  energy = -abs(np.angle(rho_c[0,1]))/(trot_time % ((np.pi)/mu))
  objective = energy - Temp*purity(rho_c)
  return objective

In [ ]:
#Find the Ground State (Coeffs)
T = 1
n_shots = 1e5
time = 26

opt_res = minimize(cost_fun_tomo, mf_coeffs, args=(time, n_shots, T), method='cobyla') 

gs_coeffs = opt_res.x
print(gs_coeffs)
result = simulator.simulate(cirq.Circuit(state_preparation(gs_coeffs)), initial_state=0)
#print(openfermion.expectation(sparse_hamiltonian, result.final_state_vector).real)
print(cirq.dirac_notation(result.final_state_vector))

T=0
print('Energy:', cost_fun_tomo(gs_coeffs, time, n_shots, T))

[-0.00360992 -0.00247431  0.01883938  0.03304771  0.05012136]
-0.03|0011⟩ - 0.03|0101⟩ - 0.02|0110⟩ + |1100⟩
Energy: -1.1186532081698366


**WAVES 2: Excited States**

In [ ]:
#Excitation operators must be put in Unitary form via UCC (Unitary Coupled Cluster)

def exc_gates(i,j,theta,qubits):
  exc_fop = openfermion.uccsd_generator([((j,i),1)], [])
  exc_qop = openfermion.jordan_wigner(exc_fop)
  exc_psum = openfermion.qubit_operator_to_pauli_sum(exc_qop, qubits)
  yield cirq.PauliSumExponential(exc_psum, exponent = theta)

exc_circ = cirq.Circuit(cirq.ops.identity_each(*qubits))
exc_circ.append(exc_gates(0,2,np.pi/2,qubits))
#exc_circ.append(exc_gates(3,1,np.pi/2,qubits))
print(exc_circ)
print(cirq.dirac_notation(exc_circ.final_state_vector(12)))

0: ───I───[Y]────────[X]───────
      │   │          │
1: ───I───[Z]────────[Z]───────
      │   │          │
2: ───I───[X]^-0.5───[Y]^0.5───
      │
3: ───I────────────────────────
-1|0110⟩


In [ ]:
#Find the Excited State (Coeffs)
T = 100
n_shots = 1e5
trot_steps = 1
time = 1

opt_res = minimize(cost_fun_tomo, mf_coeffs, args=(time, n_shots, T, trot_steps, exc_circ), method='cobyla') 

exc_coeffs = opt_res.x
print(exc_coeffs)
result = simulator.simulate(cirq.Circuit(state_preparation(exc_coeffs)), initial_state=0)
#print(openfermion.expectation(sparse_hamiltonian, result.final_state_vector).real)
print(cirq.dirac_notation(result.final_state_vector))

T=0
print('Energy:', cost_fun_tomo(exc_coeffs, time, n_shots, T, trot_steps, exc_circ))

[ 1.00239911e+00  5.08274747e-04 -5.76086040e-03  9.98378787e-01
 -1.35442138e-02]
-0.45|0101⟩ - 0.84|1010⟩ + 0.29|1100⟩
Energy: -0.5159683305920048


In [ ]:
#The same circuit will find the Ground State in the low temperature limit
T = 1
n_shots = 1e5
trot_steps = 1

opt_res = minimize(cost_fun_tomo, mf_coeffs, args=(time, n_shots, T, trot_steps, exc_circ), method='cobyla') 

print('Energy:', cost_fun_tomo(opt_res.x, time, n_shots, 0, trot_steps, exc_circ))

Energy: -1.119076438025564


In [ ]:
#Finding other excited states
exc_circ = cirq.Circuit(cirq.ops.identity_each(*qubits))
exc_circ.append(exc_gates(0,2,np.pi/2,qubits))
exc_circ.append(exc_gates(1,3,np.pi/2,qubits))

T=100
n_shots = 1e5
trot_steps = 1

opt_res = minimize(cost_fun_tomo, mf_coeffs, args=(time, n_shots, T, trot_steps, exc_circ), method='cobyla') 

exc_coeffs1 = opt_res.x
result = simulator.simulate(cirq.Circuit(state_preparation(exc_coeffs1)), initial_state=0)
#print(openfermion.expectation(sparse_hamiltonian, result.final_state_vector).real)
print(cirq.dirac_notation(result.final_state_vector))

T=0
print('Energy:', cost_fun_tomo(exc_coeffs1, time, n_shots, T, trot_steps, exc_circ))

0.06|0011⟩ + 0.06|0101⟩ - 0.01|0110⟩ + 0.04|1001⟩ + 0.04|1010⟩ + 0.99|1100⟩
Energy: -0.4680903787331588


**(1) Implement IPEA** (Iterative Phase Estimation Algorithm - Kitaev) **on the obtained ground and excited states.**

**(2) Determine the accuracy of the resulting energies.**

https://medium.com/quantum-untangled/iterative-quantum-phase-estimation-qpe-algorithms-ced794341487

https://arxiv.org/pdf/2110.02864.pdf

First IPEA is implemeneted on a test unitary 

$U(θ)|0⟩ = 1|0⟩$ 

$U(θ)|1⟩ = e^{i2πθ}|1⟩$ 

(We will need this unitary later in our final IPEA step for correcting the phase)

In [ ]:
def unitary(theta):
  unitary = np.identity(2).astype(dtype=np.complex64)
  unitary[1][1] = np.exp(2*np.pi*1j*theta)
  return unitary

"""Define a custom gate with a parameter."""
class unitary_gate(cirq.Gate):
    def __init__(self, theta):
        super(unitary_gate, self)
        self.theta = theta

    def _num_qubits_(self):
        return 1

    def _unitary_(self):
        return unitary(self.theta)

    def _circuit_diagram_info_(self, args):
        return [f"U({round(self.theta,2)})"] * self.num_qubits()

In [ ]:
n_qubits = openfermion.count_qubits(hamiltonian)
qubits = cirq.LineQubit.range(n_qubits)
control = cirq.LineQubit(-1)

circuit = cirq.Circuit(cirq.H(control))
circuit.append([unitary_gate(0.15625).on(qubits[0]).controlled_by(control)]*4)
#circuit.append(cirq.H(control))
print(circuit)
print(cirq.dirac_notation(circuit.final_state_vector(1).astype(dtype=np.complex64)))

-1: ───H───@─────────@─────────@─────────@─────────
           │         │         │         │
0: ────────U(0.16)───U(0.16)───U(0.16)───U(0.16)───
0.71|01⟩ + (-0.5-0.5j)|11⟩


In [ ]:
def ipea_circ(iteration, unitary, theta, qubit, control):
  yield [unitary.on(*qubit).controlled_by(control)] * iteration
  yield [unitary_gate(-1*theta)(control), cirq.H(control), cirq.measure(control, key='IPEA')]

In [ ]:
precision = 10
theta = 1 

circuit = cirq.Circuit([cirq.H(control), ipea_circ(precision, unitary_gate(0.2), theta, [qubits[0]], control)])
print(circuit)

-1: ───H───@────────@────────@────────@────────@────────@────────@────────@────────@────────@────────U(-1)───H───M('IPEA')───
           │        │        │        │        │        │        │        │        │        │
0: ────────U(0.2)───U(0.2)───U(0.2)───U(0.2)───U(0.2)───U(0.2)───U(0.2)───U(0.2)───U(0.2)───U(0.2)───────────────────────────


In [ ]:
def IPEA(precision, unitary, qubit, control):
  phi = 0
  for m in reversed(range(precision)):
    phi = phi/2
    circuit = cirq.Circuit(cirq.X.on(qubit))
    circuit.append(cirq.H(control))
    circuit.append(ipea_circ(2**m, unitary, phi, [qubit], control))
    res = simulator.run(circuit, repetitions=1)
    hist = res.histogram(key='IPEA')
    phi += max(hist, key = lambda k: hist[k])/2
  return phi

In [ ]:
simulator = cirq.Simulator()

phi = 0.2
precision = 10

print('phi =', IPEA(precision, unitary_gate(phi), qubits[0], control))

phi = 0.201171875


If we are not in an eigenstate of $U$ we will probabalistically find one of the eigenphases, with probabilities given by the square of the eigenstate amplitudes. In theory this will still converge to the correct eigenphase for one repition if the precision (number of bits) is high enough. However high precision is computationally expensive - To save time we average over 50 measurements, finding the probibalistic value of each bit, and run at lower precisions. 

Here we expect to find $ϕ ∼ 0.2$ for $|1⟩$ and $ϕ ∼ 0, 1$ for $|0⟩$.

In [ ]:
def IPEA(precision, unitary, qubit, control):
  phi = 0
  for m in reversed(range(precision)):
    phi = phi/2
    circuit = cirq.Circuit(cirq.H.on(qubit))
    circuit.append(cirq.H(control))
    circuit.append(ipea_circ(2**m, unitary, phi, [qubit], control))
    res = simulator.run(circuit, repetitions=50)
    hist = res.histogram(key='IPEA')
    phi += max(hist, key = lambda k: hist[k])/2
  return phi

simulator = cirq.Simulator()

phi = 0.2
precision = 10

print('phi =', IPEA(precision, unitary_gate(phi), qubits[0], control))

phi = 0.9970703125


Now we modify our IPEA circuit to apply it to the trotterized Hamiltonian

In [ ]:
def trot_unitary(trot_time):
  circ = cirq.Circuit()
  circ.append(cirq.Circuit(openfermion.simulate_trotter(qubits, hamiltonian, trot_time, n_steps=10, order=0, algorithm=trotter.LOW_RANK)))
  return cirq.unitary(circ)

"""def trot_unitary(theta):
  unitary = np.identity(16).astype(dtype=np.complex64)
  unitary[1][1] = np.exp(2*np.pi*1j*theta)
  return unitary"""

"""Define a custom gate with a parameter."""
class trot_gate(cirq.Gate):
    def __init__(self, trot_time):
        super(trot_gate, self)
        self.trot_time = trot_time % ((np.pi)/mu)

    def _num_qubits_(self):
        return 4

    def _unitary_(self):
        return trot_unitary(self.trot_time)

    def _circuit_diagram_info_(self, args):
        return [f"Trot_H(t={round(self.trot_time,2)})"] * self.num_qubits()

In [ ]:
circuit = cirq.Circuit([cirq.H(control), trot_gate(1).on(*qubits).controlled_by(control)])

print(circuit)
print(cirq.dirac_notation(circuit.final_state_vector(12)))

-1: ───H───@───────────────
           │
0: ────────Trot_H(t=1.0)───
           │
1: ────────Trot_H(t=1.0)───
           │
2: ────────Trot_H(t=1.0)───
           │
3: ────────Trot_H(t=1.0)───
0.71|01100⟩ + (-0.07-0.09j)|10011⟩ + (0.69+0.08j)|11100⟩


In [ ]:
def ipea_circ(iteration, unitary, theta, qubit, control):
  yield [unitary.on(*qubit).controlled_by(control)] * iteration
  yield [unitary_gate(-1*theta)(control), cirq.H(control), cirq.measure(control, key='IPEA')]

In [ ]:
iterations = 6
time = 1
theta = 1

gs_circ = cirq.Circuit(cirq.ops.identity_each(*qubits))

circuit = cirq.Circuit([cirq.H(control), state_preparation(gs_coeffs), gs_circ])
circuit.append(ipea_circ(iterations, trot_gate(1), theta, qubits, control))

print(circuit)
print(cirq.dirac_notation(circuit.final_state_vector(0)))

-1: ───H─────────────────────────────────────────────────────────────────────────────────────────────────────────────────@───────────────@───────────────@───────────────@───────────────@───────────────@───────────────U(-1)───H───M('IPEA')───
                                                                                                                         │               │               │               │               │               │
0: ────────X─────────────────────────────────────────────────PhISwap(0.25)^0.012───PhISwap(0.25)^0.021───G^2(0.05)───I───Trot_H(t=1.0)───Trot_H(t=1.0)───Trot_H(t=1.0)───Trot_H(t=1.0)───Trot_H(t=1.0)───Trot_H(t=1.0)───────────────────────────
                                                             │                     │                     │           │   │               │               │               │               │               │
1: ────────X───PhISwap(0.25)──────────PhISwap(0.25)──────────@─────────────────────@─────────────────────G^2(0

In [ ]:
def IPEA(precision, unitary, coeffs, exc_circ, qubit, control):
  phi = 0
  for m in reversed(range(precision)):
    phi = phi/2
    circuit = cirq.Circuit([cirq.H(control), state_preparation(coeffs), exc_circ])
    circuit.append(ipea_circ(2**m, unitary, phi, qubit, control))
    res = simulator.run(circuit, repetitions=1)
    hist = res.histogram(key='IPEA')
    phi += max(hist, key = lambda k: hist[k])/2
  return phi

In [ ]:
simulator = cirq.Simulator()

precision = 7
mu = np.max(np.abs(openfermion.eigenspectrum(fermionic_hamiltonian)))
time = 10

x = IPEA(precision, trot_gate(time), gs_coeffs, gs_circ, qubits, control)
print('phi =', x)
print('Energy =', x*(2*np.pi)/time)

phi = 0.03125
Energy = 0.019634954084936207


In [ ]:
simulator = cirq.Simulator()

precision = 7
mu = np.max(np.abs(openfermion.eigenspectrum(fermionic_hamiltonian)))
time = 10

x = IPEA(precision, trot_gate(time), exc_coeffs, exc_circ, qubits, control)
print('phi =', x)
print('Energy =', x*(2*np.pi)/time)
print('Fidelity =', (x*(2*np.pi)/time)/np.abs(openfermion.eigenspectrum(fermionic_hamiltonian)[2]))

phi = 0.8671875
Energy = 0.5448699758569797
Fidelity = 1.0114354654638522


In [ ]:
print('Eigenspectrum of H2 in FermionOperator: \n',
      openfermion.eigenspectrum(fermionic_hamiltonian))

Eigenspectrum of H2 in FermionOperator: 
 [-1.13727017 -0.53870958 -0.53870958 -0.53247901 -0.53247901 -0.53247901
 -0.44698572 -0.44698572 -0.16990139  0.23780527  0.23780527  0.35243413
  0.35243413  0.47983611  0.71375399  0.92010671]


In [ ]:
simulator = cirq.Simulator()

precision = 8
time = 1

x = IPEA(precision, unitary_gate(phi), gs_coeffs, gs_circ, [qubits[0]], control)
#print('phi =', IPEA(precision, unitary_gate(phi), gs_coeffs, np.full(0,5), [qubits[0]], control))
#print('phi =', IPEA(precision, unitary_gate(phi), np.full(5,0), gs_circ, [qubits[0]], control))
print('phi =', x)
print('E=',x*2*np.pi/t)

phi = 0.30078125
E= 1.88986433067511
